In [1]:
int2op = {"00": "STOP",
"01": "ADD",
"02": "MUL",
"03": "SUB",
"04": "DIV",
"05": "SDIV",
"06": "MOD",
"07":"SMOD",
"08":"ADDMOD",
"09":"MULMOD",
"0a":"EXP",
"0b":"SIGNEXTEND",
"10":"LT",
"11":"GT",
"12":"SLT",
"13":"SGT",
"14":"EQ",
"15":"ISZERO",
"16":"AND",
"17":"OR",
"18":"XOR",
"19":"NOT",
"1a":"BYTE",
"1b":"SHL",
"1c": "SHR",
"1d": "SAR",
"20":"SHA3",
"30":"ADDRESS",
"31":"BALANCE",
"32":"ORIGIN",
"33":"CALLER",
"34":"CALLVALUE",
"35":"CALLDATALOAD",
"36":"CALLDATASIZE",
"37":"CALLDATACOPY",
"38":"CODESIZE",
"39":"CODECOPY",
"3a":"GASPRICE",
"3b":"EXTCODESIZE",
"3c":"EXTCODECOPY",
"3d":"RETURNDATASIZE",
"3e":"RETURNDATACOPY",
"3f": "EXTCODEHASH",
"40":"BLOCKHASH",
"41":"COINBASE",
"42":"TIMESTAMP",
"43":"NUMBER",
"44":"DIFFICULTY",
"45":"GASLIMIT",
"46": "CHAINID",
"47": "SELFBALANCE",
"50":"POP",
"51":"MLOAD",
"52":"MSTORE",
"53":"MSTORE8",
"54":"SLOAD",
"55":"SSTORE",
"56":"JUMP",
"57":"JUMPI",
"58":"PC",
"59":"MSIZE",
"5a":"GAS",
"5b":"JUMPDEST",
"60":"PUSH1",
"61":"PUSH2",
"62":"PUSH3",
"63":"PUSH4",
"64":"PUSH5",
"65":"PUSH6",
"66":"PUSH7",
"67":"PUSH8",
"68":"PUSH9",
"69":"PUSH10",
"6a":"PUSH11",
"6b":"PUSH12",
"6c":"PUSH13",
"6d":"PUSH14",
"6e":"PUSH15",
"6f":"PUSH16",
"70":"PUSH17",
"71":"PUSH18",
"72":"PUSH19",
"73":"PUSH20",
"74":"PUSH21",
"75":"PUSH22",
"76":"PUSH23",
"77":"PUSH24",
"78":"PUSH25",
"79":"PUSH26",
"7a":"PUSH27",
"7b":"PUSH28",
"7c":"PUSH29",
"7d":"PUSH30",
"7e":"PUSH31",
"7f":"PUSH32",
"80":"DUP1",
"81":"DUP2",
"82":"DUP3",
"83":"DUP4",
"84":"DUP5",
"85":"DUP6",
"86":"DUP7",
"87":"DUP8",
"88":"DUP9",
"89":"DUP10",
"8a":"DUP11",
"8b":"DUP12",
"8c":"DUP13",
"8d":"DUP14",
"8e":"DUP15",
"8f":"DUP16",
"90":"SWAP1",
"91":"SWAP2",
"92":"SWAP3",
"93":"SWAP4",
"94":"SWAP5",
"95":"SWAP6",
"96":"SWAP7",
"97":"SWAP8",
"98":"SWAP9",
"99":"SWAP10",
"9a":"SWAP11",
"9b":"SWAP12",
"9c":"SWAP13",
"9d":"SWAP14",
"9e":"SWAP15",
"9f":"SWAP16",
"a0":"LOG0",
"a1":"LOG1",
"a2":"LOG2",
"a3":"LOG3",
"a4":"LOG4",
"b0":"PUSH",
"b1":"DUP",
"b2":"SWAP",
"f0":"CREATE",
"f1":"CALL",
"f2":"CALLCODE",
"f3":"RETURN",
"f4":"DELEGATECALL",
"f5":"CREATE2",
"fa":"STATICCALL",
"fd":"REVERT",
"fe":"INVALID",
"ff":"SELFDESTRUCT"}

In [2]:
from datasets import Dataset
import pandas as pd
import re

/home/bombbom/Documents/NLP_in_Detection_System/env_nlp/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
data = pd.read_pickle("/home/bombbom/Documents/NLP_in_Detection_System/dataset_example/labeled_SBW_datasets.pkl")
data

,address,source_code,categories,lines
0,0x21e13cb3f3f26f92a62ac7adab4093e8997d1fb1,pragma solidity ^0.4.18;\n\n// ---------------...,"['Other', 'arithmetic', 'arithmetic']","[154, 141, 103]"
1,0x344005c29af957567f0b40950b425ed018b92170,pragma solidity ^0.4.21;\n\n\ncontract DSMath...,"['arithmetic', 'reentrancy', 'arithmetic', 'ar...","[201, 16, 209, 176, 146, 154, 29]"
2,0x44e320110176c11c93e116f6770f13d96deded43,pragma solidity ^0.4.20;\n\n\ncontract Ownable...,['arithmetic'],"[82, 46]"
3,0x5ecd84482176db90bb741ddc8c2f9ccc290e29ce,pragma solidity ^0.4.8;\ncontract Token{\n ...,"['arithmetic', 'unchecked_low_calls', 'arithme...","[99, 38, 78, 48, 49, 60, 93]"
4,0xcaf187eb618d2335b4130d784a697be96f4b07b9,pragma solidity 0.4.15;\n\ncontract RegistryIC...,['Other'],"[80, 28, 63]"
...,...,...,...,...
47326,0x7f650f3b231d3a32c2b0e2940e870acdd4aa9961,pragma solidity ^0.4.24;\n\n\n\n\n/**\n * @tit...,"['reentrancy', 'arithmetic', 'time_manipulatio...","[258, 264, 245, 253, 287]"
47327,0xc73f2474001ad1d6aed615af53631148cf98de6b,pragma solidity ^0.4.18;\n\n// ---------------...,"['Other', 'arithmetic', 'arithmetic']","[200, 218, 128, 102]"
47328,0xb2105f178abe620ddbb86a70afa94bfa9daa4d01,pragma solidity ^0.4.23;\n\nlibrary SafeMath {...,"['arithmetic', 'arithmetic', 'front_running']","[124, 20, 205]"
47329,0xe2d4b960d0c639633582cddef57528461e62083d,pragma solidity ^0.4.23;\n\n\n/// @title Multi...,"['arithmetic', 'Other', 'unchecked_low_calls',...","[140, 142, 23, 282, 410, 157, 287, 296, 302, 3..."


In [4]:
# data = data[['address', 'source_code', 'categories']]

In [4]:
import re
def get_version(source_code):
    x = re.findall("[0][.][0-9][.][0-9]*[0-9]", source_code)
    if len(x)>0:
        return x[0]
    return "0.4.25"

In [5]:
data['version'] = data.source_code.apply(get_version)

In [7]:
def change(version):
    if version =='0.4.99' :
        return '0.5.0'

In [8]:
data['version'].unique()

array(['0.4.18', '0.4.21', '0.4.20', '0.4.8', '0.4.15', '0.4.17',
       '0.4.24', '0.4.4', '0.4.19', '0.4.16', '0.4.25', '0.4.13',
       '0.4.99', '0.5.0', '0.4.22', '0.4.11', '0.4.23', '0.5.2', '0.5.7',
       '0.5.6', '0.5.4', '0.4.0', '0.5.1', '0.4.2', '0.4.3', '0.4.9',
       '0.5.5', '0.4.7', '0.4.10', '0.5.00', '0.5.8', '0.4.14', '0.4.12',
       '0.7.1', '0.5.9', '0.4.6', '0.1.4', '0.0.1', '0.5.3', '0.3.14',
       '0.4.26', '0.1.0', '0.5.10', '0.2.1', '0.4.1', '0.3.6', '0.3.5',
       '0.9.9', '0.0.2', '0.4.5', '0.2.4', '0.3.1', '0.1.5', '0.1.2',
       '0.3.3', '0.1.3', '0.1.1', '0.3.2', '0.6.0'], dtype=object)

In [9]:
value = [ '0.4.8', '0.4.4',  '0.4.0',  '0.4.2', '0.4.3', '0.4.9', '0.4.7', '0.4.10', '0.5.00', '0.4.6', '0.1.4', '0.0.1',  '0.3.14','0.1.0',  '0.2.1', '0.4.1', '0.3.6', '0.3.5','0.9.9', '0.0.2', '0.4.5', '0.2.4', '0.3.1', '0.1.5', '0.1.2','0.3.3', '0.1.3', '0.1.1', '0.3.2']

def filter_rows_by_values(df, col, values):
       df = data[~data[col].isin(values)]
       return df
       

In [10]:
data = filter_rows_by_values(data, "version", value)

In [11]:
data.reset_index(drop = "True")

,address,source_code,categories,lines,version
0,0x21e13cb3f3f26f92a62ac7adab4093e8997d1fb1,pragma solidity ^0.4.18;\n\n// ---------------...,"['Other', 'arithmetic', 'arithmetic']","[154, 141, 103]",0.4.18
1,0x344005c29af957567f0b40950b425ed018b92170,pragma solidity ^0.4.21;\n\n\ncontract DSMath...,"['arithmetic', 'reentrancy', 'arithmetic', 'ar...","[201, 16, 209, 176, 146, 154, 29]",0.4.21
2,0x44e320110176c11c93e116f6770f13d96deded43,pragma solidity ^0.4.20;\n\n\ncontract Ownable...,['arithmetic'],"[82, 46]",0.4.20
3,0xcaf187eb618d2335b4130d784a697be96f4b07b9,pragma solidity 0.4.15;\n\ncontract RegistryIC...,['Other'],"[80, 28, 63]",0.4.15
4,0x2ab58a02c4d26bcf3c6116f62ec5c1a6d4c7a53f,pragma solidity ^0.4.18;\n\n// File: openzeppe...,"['arithmetic', 'front_running', 'Other', 'reen...","[37, 166, 199, 103, 167, 175, 80, 178, 84, 86,...",0.4.18
...,...,...,...,...,...
42829,0xc2396239e59d07bd726e94411413a26152e8c733,pragma solidity ^0.4.18;\n\n// ---------------...,"['Other', 'arithmetic', 'arithmetic']","[129, 219, 201]",0.4.18
42830,0x7f650f3b231d3a32c2b0e2940e870acdd4aa9961,pragma solidity ^0.4.24;\n\n\n\n\n/**\n * @tit...,"['reentrancy', 'arithmetic', 'time_manipulatio...","[258, 264, 245, 253, 287]",0.4.24
42831,0xc73f2474001ad1d6aed615af53631148cf98de6b,pragma solidity ^0.4.18;\n\n// ---------------...,"['Other', 'arithmetic', 'arithmetic']","[200, 218, 128, 102]",0.4.18
42832,0xb2105f178abe620ddbb86a70afa94bfa9daa4d01,pragma solidity ^0.4.23;\n\nlibrary SafeMath {...,"['arithmetic', 'arithmetic', 'front_running']","[124, 20, 205]",0.4.23


In [12]:
data.version = data.version.apply(change)

/tmp/ipykernel_105203/1490153070.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.version = data.version.apply(change)


In [21]:
import solcx
solcx.install_solc('0.4.11')

Version('0.4.11')

In [13]:
import os
import solcx
def compiler(input):
    version = get_version(input)
    token = ""
    
    try: 
        solcx.install_solc(version)
        a = solcx.compile_source(
            input,
            output_values=["opcodes"],
            solc_version=version
        )
        
        for key, value in a.items():
            for k in value:
                # print(value[k])
                # print(len(value[k]))
                # token = value[k].split()
                # print(len(token))
                token = value[k]
    except:
        token=""
    
    return token

In [14]:
data['opcode'] = data.source_code.apply(compiler)

/tmp/ipykernel_105203/1518998255.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['opcode'] = data.source_code.apply(compiler)


In [9]:
data.to_pickle("data_full.pkl")

In [10]:
data

,source_code,version
0,PUSH1 0x60 PUSH1 0x40 MSTORE CALLVALUE ISZERO ...,None
1,PUSH1 0x60 PUSH1 0x40 MSTORE CALLVALUE ISZERO ...,None
2,PUSH1 0x60 PUSH1 0x40 MSTORE CALLVALUE ISZERO ...,None
4,PUSH1 0x60 PUSH1 0x40 MSTORE CALLVALUE ISZERO ...,None
5,PUSH1 0x60 PUSH1 0x40 MSTORE PUSH1 0x40 DUP1 M...,None
...,...,...
47325,PUSH1 0x60 PUSH1 0x40 MSTORE CALLVALUE ISZERO ...,None
47326,PUSH1 0x4C PUSH1 0x2C PUSH1 0xB DUP3 DUP3 DUP3...,None
47327,PUSH1 0x60 PUSH1 0x40 MSTORE CALLVALUE ISZERO ...,None
47328,PUSH1 0x4C PUSH1 0x2C PUSH1 0xB DUP3 DUP3 DUP3...,None


In [11]:
data=data[['source_code']]

In [14]:
data = data.reset_index(drop=True)

In [7]:
data =pd.read_pickle("/home/bombbom/Documents/NLP_in_Detection_System/data.pkl")

In [15]:
data

,source_code
0,PUSH1 0x60 PUSH1 0x40 MSTORE CALLVALUE ISZERO ...
1,PUSH1 0x60 PUSH1 0x40 MSTORE CALLVALUE ISZERO ...
2,PUSH1 0x60 PUSH1 0x40 MSTORE CALLVALUE ISZERO ...
3,PUSH1 0x60 PUSH1 0x40 MSTORE CALLVALUE ISZERO ...
4,PUSH1 0x60 PUSH1 0x40 MSTORE PUSH1 0x40 DUP1 M...
...,...
42829,PUSH1 0x60 PUSH1 0x40 MSTORE CALLVALUE ISZERO ...
42830,PUSH1 0x4C PUSH1 0x2C PUSH1 0xB DUP3 DUP3 DUP3...
42831,PUSH1 0x60 PUSH1 0x40 MSTORE CALLVALUE ISZERO ...
42832,PUSH1 0x4C PUSH1 0x2C PUSH1 0xB DUP3 DUP3 DUP3...


In [16]:
data = Dataset.from_pandas(data)

In [17]:
def get_training_corpus():
    return (
        data[i : i + 1000]["source_code"]
        for i in range(0, len(data), 1000)
    )


training_corpus = get_training_corpus()

In [18]:
from transformers import AutoTokenizer

old_tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [19]:
example = '''PUSH1 0x80 PUSH1 0x40 MSTORE CALLVALUE DUP1'''

tokens = old_tokenizer.tokenize(example)
tokens

['push',
 '##1',
 '0',
 '##x',
 '##80',
 'push',
 '##1',
 '0',
 '##x',
 '##40',
 'ms',
 '##tore',
 'call',
 '##val',
 '##ue',
 'du',
 '##p',
 '##1']

In [20]:
old_tokenizer.add_tokens(list(int2op.values()))

145

In [21]:
tokenizer = old_tokenizer.train_new_from_iterator(training_corpus, 52000)


In [22]:
tokens = tokenizer.tokenize(example)
tokens

['push1', '0x80', 'push1', '0x40', 'mstore', 'callvalue', 'dup1']

In [23]:
tokenizer.save_pretrained("/home/bombbom/Documents/NLP_in_Detection_System/save_model/tokenizer_opcode/")

('/home/bombbom/Documents/NLP_in_Detection_System/save_model/tokenizer_opcode/tokenizer_config.json',
 '/home/bombbom/Documents/NLP_in_Detection_System/save_model/tokenizer_opcode/special_tokens_map.json',
 '/home/bombbom/Documents/NLP_in_Detection_System/save_model/tokenizer_opcode/vocab.txt',
 '/home/bombbom/Documents/NLP_in_Detection_System/save_model/tokenizer_opcode/added_tokens.json',
 '/home/bombbom/Documents/NLP_in_Detection_System/save_model/tokenizer_opcode/tokenizer.json')

# Pretrain

In [24]:
df=pd.read_pickle("/home/bombbom/Documents/NLP_in_Detection_System/data.pkl")

In [25]:
df =df[['source_code']]

In [26]:
df

,source_code
0,PUSH1 0x60 PUSH1 0x40 MSTORE CALLVALUE ISZERO ...
1,PUSH1 0x60 PUSH1 0x40 MSTORE CALLVALUE ISZERO ...
2,PUSH1 0x60 PUSH1 0x40 MSTORE CALLVALUE ISZERO ...
4,PUSH1 0x60 PUSH1 0x40 MSTORE CALLVALUE ISZERO ...
5,PUSH1 0x60 PUSH1 0x40 MSTORE PUSH1 0x40 DUP1 M...
...,...
47325,PUSH1 0x60 PUSH1 0x40 MSTORE CALLVALUE ISZERO ...
47326,PUSH1 0x4C PUSH1 0x2C PUSH1 0xB DUP3 DUP3 DUP3...
47327,PUSH1 0x60 PUSH1 0x40 MSTORE CALLVALUE ISZERO ...
47328,PUSH1 0x4C PUSH1 0x2C PUSH1 0xB DUP3 DUP3 DUP3...


In [28]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df,test_size=0.3, random_state=20)

In [29]:
train, val = train_test_split(train,test_size=0.15, random_state=20)

In [30]:
data_train = Dataset.from_pandas(train)
data_test = Dataset.from_pandas(test)
data_val = Dataset.from_pandas(val)

In [31]:
def preprocess_function(examples):
    return tokenizer([" ".join(x) for x in examples["source_code"]], padding = "max_length", truncation=True)

block_size = 128

def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # Split by chunks of block_size.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result


In [32]:
tokenized_data_train = data_train.map(preprocess_function, batched = True, num_proc=4, remove_columns = data_train.column_names)
tokenized_data_val = data_val.map(preprocess_function, batched = True, num_proc=4, remove_columns = data_val.column_names)


In [33]:
lm_dataset_train = tokenized_data_train.map(group_texts, batched=True, num_proc=4 )
lm_dataset_val = tokenized_data_val.map(group_texts, batched=True, num_proc=4)

In [36]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)
from transformers import AutoModelForMaskedLM
model = AutoModelForMaskedLM.from_pretrained("distilbert-base-uncased")
print(model)
# %%
len(tokenizer)
## Resize 
model.resize_token_embeddings(len(tokenizer))
print(model)

DistilBertForMaskedLM(
  (activation): GELUActivation()
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.

In [37]:
## Step 5: Setup training_args 
from transformers import Trainer, TrainingArguments
training_args = TrainingArguments(
    output_dir="pre-train-mlm_opcode",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=10,
    weight_decay=0.01,
    save_steps=10000,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset_train,
    eval_dataset=lm_dataset_val,
    data_collator=data_collator,
)


In [ ]:
train.train()

In [ ]:
train.save()